In [1]:
import os
os.chdir('../..')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

### Preparing data
We'll predict either the customer'll:
- apply for internet connection
- use fiber optics
- use dsl

In [3]:
df = pd.read_csv('datasets/customer-churn/encoded-unscaled.csv')
df.head(1)

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber optic,...,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,1,29.85,29.85,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [4]:
df.shape

(7032, 24)

In [8]:
df2 = pd.read_csv('datasets/customer-churn/telco-customer-churn.csv')
df2.head(1)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [9]:
df2.drop(index=df2[df2['TotalCharges'] == ' '].index, inplace=True, axis=1)

In [10]:
df2.shape

(7032, 21)

In [11]:
df['InternetService'] = df2['InternetService']

In [12]:
df.head(1)

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber optic,...,StreamingTV_Yes,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes,InternetService
0,1,29.85,29.85,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,DSL


In [13]:
df.columns

Index(['tenure', 'MonthlyCharges', 'TotalCharges', 'SeniorCitizen',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_Yes', 'OnlineBackup_Yes',
       'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes',
       'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year',
       'PaperlessBilling_Yes', 'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'Churn_Yes', 'InternetService'],
      dtype='object')

In [14]:
df = df.drop(['InternetService_Fiber optic', 'InternetService_No'], axis=1)
df.shape

(7032, 23)

In [15]:
pd.set_option('display.max_columns', None)

In [16]:
df.head(2)

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes,InternetService
0,1,29.85,29.85,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,DSL
1,34,56.95,1889.50,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,DSL


In [17]:
df['InternetService'].value_counts()

InternetService
Fiber optic    3089
DSL            2413
No             1519
Name: count, dtype: int64

In [18]:
df['InternetService'] = df['InternetService'].map({'No': 0, 'DSL': 1, 'Fiber optic': 2})

In [19]:
df.dtypes

tenure                                     int64
MonthlyCharges                           float64
TotalCharges                             float64
SeniorCitizen                              int64
gender_Male                                int64
Partner_Yes                                int64
Dependents_Yes                             int64
PhoneService_Yes                           int64
MultipleLines_Yes                          int64
OnlineSecurity_Yes                         int64
OnlineBackup_Yes                           int64
DeviceProtection_Yes                       int64
TechSupport_Yes                            int64
StreamingTV_Yes                            int64
StreamingMovies_Yes                        int64
Contract_One year                          int64
Contract_Two year                          int64
PaperlessBilling_Yes                       int64
PaymentMethod_Credit card (automatic)      int64
PaymentMethod_Electronic check             int64
PaymentMethod_Mailed

In [21]:
df.isnull().sum()

tenure                                    0
MonthlyCharges                            0
TotalCharges                              0
SeniorCitizen                             0
gender_Male                               0
Partner_Yes                               0
Dependents_Yes                            0
PhoneService_Yes                          0
MultipleLines_Yes                         0
OnlineSecurity_Yes                        0
OnlineBackup_Yes                          0
DeviceProtection_Yes                      0
TechSupport_Yes                           0
StreamingTV_Yes                           0
StreamingMovies_Yes                       0
Contract_One year                         0
Contract_Two year                         0
PaperlessBilling_Yes                      0
PaymentMethod_Credit card (automatic)     0
PaymentMethod_Electronic check            0
PaymentMethod_Mailed check                0
Churn_Yes                                 0
InternetService                 

In [23]:
df.dropna(axis=0, inplace=True)

In [24]:
df.head()

,tenure,MonthlyCharges,TotalCharges,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes,InternetService
0,1,29.85,29.85,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1.0
1,34,56.95,1889.50,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1.0
2,2,53.85,108.15,0,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,1,1.0
3,45,42.30,1840.75,0,1,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1.0
4,2,70.70,151.65,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,2.0


In [26]:
df['InternetService'].value_counts()

InternetService
2.0    3089
1.0    2413
0.0    1519
Name: count, dtype: int64

In [32]:
def build_c_model(data: pd.DataFrame, target_name: str, model_module, model_name: str, multinomial='auto', test_size: float= 0.2) -> None:
    x_train, x_test, y_train, y_test = train_test_split(data.drop(target_name, axis=1), data[target_name], test_size=test_size, random_state=40)
    print('Splitted')
    model = model_module(multi_class=multinomial)
    model.fit(x_train, y_train)
    print('Fitted')
    y_pred = model.predict(x_test)
    report = classification_report(y_test, y_pred)
    print(f'Model: {model_name} \n Classification report: \n{report}')

In [38]:
build_c_model(df, 'InternetService', LogisticRegression, 'LogisticRegression', 'multinomial', .2)

Splitted
Fitted
Model: LogisticRegression 
 Classification report: 
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       291
         1.0       0.44      0.09      0.15       487
         2.0       0.45      0.95      0.61       627

    accuracy                           0.45      1405
   macro avg       0.30      0.34      0.25      1405
weighted avg       0.35      0.45      0.32      1405



c:\Users\Dmm\anaconda3\envs\innoquest-ml\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\Dmm\anaconda3\envs\innoquest-ml\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Dmm\anaconda3\envs\innoquest-ml\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 